In [1]:
# install.packages("sp")
# install.packages("rgdal")
# install.packages("rgeos")
# install.packages("ggmap")
# install.packages("tmap")
# install.packages("raster")
# install.packages("spdep")
# install.packages("gstat")
# install.packages("spgwr")
# install.packages("GWmodel")

library(dplyr)
library(tidyr)
library(stringr)
library(sp)
library(rgdal)
library(rgeos)
library(tmap)
library(raster)
library(spdep)
library(gstat)
library(spgwr)
library(GWmodel)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


rgdal: version: 1.5-16, (SVN revision 1050)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.1.2, released 2020/07/07
Path to GDAL shared files: /usr/local/gdal-3.1.2/share/gdal
GDAL binary built with GEOS: FALSE 
Loaded PROJ runtime: Rel. 6.1.0, May 15th, 2019, [PJ_VERSION: 610]
Path to PROJ shared files: /usr/local/proj-6.1.0/share/proj
Linking to sp version:1.4-2
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.

rgeos version: 0.5-3, (SVN revision 634)
 GEOS runtime version: 3.8.0-CAPI-1.13.1 
 Linking to sp version: 1.4-2 
 Polygon checking: TRUE 



Attaching package: ‘raster’


The following object is masked from ‘package:tidyr’:

    ext

## 연도별 weather 데이터 불러오기

세계기상기구(WMO)는 기온, 강수량 등의 기후요소가 평년값에 비해 현저히 높거나 낮은 수치(90 퍼센타일 또는 10 퍼센타일 미만의 범위)를 이상기후로 정의하고 있다.<br/>
이 기준에 따라서 2017~2019년의 weather 데이터의 이상치를 분류하도록 한다.

- 기온과 풍속, 습도, 지면 온도의 데이터 전처리: 1년 별로 비교해서 이상치(하위 10% 미만, 상위 10% 초과) 안에 포함되는 경우 1, 포함되지 않는 경우에는 0
- 강수량, 적설량 데이터 전처리: 당해 연도의 상위 10% 백분위 수에 포함되는 경우 1, 포함되지 않는 경우에는 0
- 안개 데이터 전처리: 안개는 지속 시간이 조금이라도 있으면 1, 없으면 0

### 2017년의 weather 데이터 처리

In [2]:
weather_2017 <- read.csv('weather_2017.csv')

In [3]:
temp_average <- c()
qt <- quantile(weather_2017[, names(weather_2017)[3]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_average <- ifelse(weather_2017[, names(weather_2017)[3]] < qt[2] | weather_2017[, names(weather_2017)[3]] > qt[3], 1, 0)

temp_min <- c()
qt <- quantile(weather_2017[, names(weather_2017)[4]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_min <- ifelse(weather_2017[, names(weather_2017)[4]] < qt[2] | weather_2017[, names(weather_2017)[4]] > qt[3], 1, 0)

temp_max <- c()
qt <- quantile(weather_2017[, names(weather_2017)[5]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_max <- ifelse(weather_2017[, names(weather_2017)[5]] < qt[2] | weather_2017[, names(weather_2017)[5]] > qt[3], 1, 0)

wind_max <- c()
qt <- quantile(weather_2017[, names(weather_2017)[7]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_max <- ifelse(weather_2017[, names(weather_2017)[7]] < qt[2] | weather_2017[, names(weather_2017)[7]] > qt[3], 1, 0)

wind_average <- c()
qt <- quantile(weather_2017[, names(weather_2017)[9]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_average <- ifelse(weather_2017[, names(weather_2017)[9]] < qt[2] | weather_2017[, names(weather_2017)[9]] > qt[3], 1, 0)

humidity_average <- c()
qt <- quantile(weather_2017[, names(weather_2017)[10]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
humidity_average <- ifelse(weather_2017[, names(weather_2017)[10]] < qt[2] | weather_2017[, names(weather_2017)[10]] > qt[3], 1, 0)

road_average <- c()
qt <- quantile(weather_2017[, names(weather_2017)[12]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
road_average <- ifelse(weather_2017[, names(weather_2017)[12]] < qt[2] | weather_2017[, names(weather_2017)[12]] > qt[3], 1, 0)

In [4]:
weather_2017$평균온도 <- temp_average
weather_2017$최저온도 <- temp_min
weather_2017$최고온도 <- temp_max
weather_2017$최대풍속 <- wind_max
weather_2017$평균풍속 <- wind_average
weather_2017$평균습도 <- humidity_average
weather_2017$평균지면온도 <- road_average

In [5]:
# NA 값은 이상치가 없는 경우이므로 0으로 처리
weather_2017[, names(weather_2017)[6]] <- ifelse(is.na(weather_2017[, names(weather_2017)[6]]), 0, weather_2017[, names(weather_2017)[6]])
weather_2017[, names(weather_2017)[11]] <- ifelse(is.na(weather_2017[, names(weather_2017)[11]]), 0, weather_2017[, names(weather_2017)[11]])

In [6]:
day_rain <- c()
qt <- quantile(weather_2017[, names(weather_2017)[6]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_rain <- ifelse(weather_2017[, names(weather_2017)[6]] > qt[3], 1, 0)

day_snow <- c()
qt <- quantile(weather_2017[, names(weather_2017)[11]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_snow <- ifelse(weather_2017[, names(weather_2017)[11]] > qt[3], 1, 0)

In [7]:
weather_2017$강수량 <- day_rain
weather_2017$적설량 <- day_snow
weather_2017$안개시간 <- ifelse( is.na(weather_2017[, names(weather_2017)[13]]), 0, 1)

In [8]:
weather_2017_arrange <- weather_2017[,c(2,14:ncol(weather_2017))] 
dim(weather_2017_arrange)
weather_2017_arrange %>% head()

[1] 365  11

,일시,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2017-01-01,0,0,0,0,1,0,0,0,0,0
2,2017-01-02,0,0,0,0,0,0,0,0,0,0
3,2017-01-03,0,0,0,0,0,0,0,0,0,0
4,2017-01-04,0,0,0,0,0,0,0,0,0,0
5,2017-01-05,0,0,0,0,0,0,0,0,0,0
6,2017-01-06,0,0,0,0,0,0,0,0,0,0


### 2018년의 weather 데이터 처리

In [9]:
weather_2018 <- read.csv('weather_2018.csv')

In [10]:
temp_average <- c()
qt <- quantile(weather_2018[, names(weather_2018)[3]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_average <- ifelse(weather_2018[, names(weather_2018)[3]] < qt[2] | weather_2018[, names(weather_2018)[3]] > qt[3], 1, 0)

temp_min <- c()
qt <- quantile(weather_2018[, names(weather_2018)[4]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_min <- ifelse(weather_2018[, names(weather_2018)[4]] < qt[2] | weather_2018[, names(weather_2018)[4]] > qt[3], 1, 0)

temp_max <- c()
qt <- quantile(weather_2018[, names(weather_2018)[5]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_max <- ifelse(weather_2018[, names(weather_2018)[5]] < qt[2] | weather_2018[, names(weather_2018)[5]] > qt[3], 1, 0)

wind_max <- c()
qt <- quantile(weather_2018[, names(weather_2018)[7]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_max <- ifelse(weather_2018[, names(weather_2018)[7]] < qt[2] | weather_2018[, names(weather_2018)[7]] > qt[3], 1, 0)

wind_average <- c()
qt <- quantile(weather_2018[, names(weather_2018)[9]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_average <- ifelse(weather_2018[, names(weather_2018)[9]] < qt[2] | weather_2018[, names(weather_2018)[9]] > qt[3], 1, 0)

humidity_average <- c()
qt <- quantile(weather_2018[, names(weather_2018)[10]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
humidity_average <- ifelse(weather_2018[, names(weather_2018)[10]] < qt[2] | weather_2018[, names(weather_2018)[10]] > qt[3], 1, 0)

road_average <- c()
qt <- quantile(weather_2018[, names(weather_2018)[12]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
road_average <- ifelse(weather_2018[, names(weather_2018)[12]] < qt[2] | weather_2018[, names(weather_2018)[12]] > qt[3], 1, 0)

In [11]:
weather_2018$평균온도 <- temp_average
weather_2018$최저온도 <- temp_min
weather_2018$최고온도 <- temp_max
weather_2018$최대풍속 <- wind_max
weather_2018$평균풍속 <- wind_average
weather_2018$평균습도 <- humidity_average
weather_2018$평균지면온도 <- road_average

In [12]:
weather_2018[, names(weather_2018)[6]] <- ifelse(is.na(weather_2018[, names(weather_2018)[6]]), 0, weather_2018[, names(weather_2018)[6]])
weather_2018[, names(weather_2018)[11]] <- ifelse(is.na(weather_2018[, names(weather_2018)[11]]), 0, weather_2018[, names(weather_2018)[11]])

In [13]:
day_rain <- c()
qt <- quantile(weather_2018[, names(weather_2018)[6]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_rain <- ifelse(weather_2018[, names(weather_2018)[6]] > qt[3], 1, 0)

day_snow <- c()
qt <- quantile(weather_2018[, names(weather_2018)[11]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_snow <- ifelse(weather_2018[, names(weather_2018)[11]] > qt[3], 1, 0)

In [14]:
weather_2018$강수량 <- day_rain
weather_2018$적설량 <- day_snow
weather_2018$안개시간 <- ifelse( is.na(weather_2018[, names(weather_2018)[13]]), 0, 1)

In [15]:
weather_2018_arrange <- weather_2018[,c(2,14:ncol(weather_2018))]
dim(weather_2018_arrange)
weather_2018_arrange %>% head()

[1] 365  11

,일시,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2018-01-01,0,0,0,0,0,0,0,0,0,0
2,2018-01-02,0,0,0,0,0,0,1,0,0,0
3,2018-01-03,1,0,1,0,0,0,1,0,0,0
4,2018-01-04,1,0,1,1,0,0,1,0,0,0
5,2018-01-05,0,0,1,0,0,0,0,0,0,0
6,2018-01-06,0,0,0,0,0,0,0,0,0,0


### 2019년의 weather 데이터 처리

In [16]:
weather_2019 <- read.csv('weather_2019.csv')

In [17]:
qt <- quantile(weather_2019[, names(weather_2017)[3]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_average <- ifelse(weather_2019[, names(weather_2019)[3]] < qt[2] | weather_2019[, names(weather_2019)[3]] > qt[3], 1, 0)

temp_min <- c()
qt <- quantile(weather_2019[, names(weather_2019)[4]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_min <- ifelse(weather_2019[, names(weather_2019)[4]] < qt[2] | weather_2019[, names(weather_2019)[4]] > qt[3], 1, 0)

temp_max <- c()
qt <- quantile(weather_2019[, names(weather_2019)[5]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
temp_max <- ifelse(weather_2019[, names(weather_2019)[5]] < qt[2] | weather_2019[, names(weather_2019)[5]] > qt[3], 1, 0)

wind_max <- c()
qt <- quantile(weather_2019[, names(weather_2019)[7]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_max <- ifelse(weather_2019[, names(weather_2019)[7]] < qt[2] | weather_2019[, names(weather_2019)[7]] > qt[3], 1, 0)

wind_average <- c()
qt <- quantile(weather_2019[, names(weather_2019)[9]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
wind_average <- ifelse(weather_2019[, names(weather_2019)[9]] < qt[2] | weather_2019[, names(weather_2019)[9]] > qt[3], 1, 0)

humidity_average <- c()
qt <- quantile(weather_2019[, names(weather_2019)[10]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
humidity_average <- ifelse(weather_2019[, names(weather_2019)[10]] < qt[2] | weather_2019[, names(weather_2019)[10]] > qt[3], 1, 0)

road_average <- c()
qt <- quantile(weather_2019[, names(weather_2019)[12]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
road_average <- ifelse(weather_2019[, names(weather_2019)[12]] < qt[2] | weather_2019[, names(weather_2019)[12]] > qt[3], 1, 0)

In [18]:
weather_2019$평균온도 <- temp_average
weather_2019$최저온도 <- temp_min
weather_2019$최고온도 <- temp_max
weather_2019$최대풍속 <- wind_max
weather_2019$평균풍속 <- wind_average
weather_2019$평균습도 <- humidity_average
weather_2019$평균지면온도 <- road_average

In [19]:
weather_2019[, names(weather_2019)[6]] <- ifelse(is.na(weather_2019[, names(weather_2019)[6]]), 0, weather_2019[, names(weather_2019)[6]])
weather_2019[, names(weather_2019)[11]] <- ifelse(is.na(weather_2019[, names(weather_2019)[11]]), 0, weather_2019[, names(weather_2019)[11]])

In [20]:
day_rain <- c()
qt <- quantile(weather_2019[, names(weather_2019)[6]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_rain <- ifelse(weather_2019[, names(weather_2019)[6]] > qt[3], 1, 0)

day_snow <- c()
qt <- quantile(weather_2019[, names(weather_2019)[11]], probs=c(0, 0.1, 0.9, 1), na.rm=T)
day_snow <- ifelse(weather_2019[, names(weather_2019)[11]] > qt[3], 1, 0)

In [21]:
weather_2019$강수량 <- day_rain
weather_2019$적설량 <- day_snow
weather_2019$안개시간 <- ifelse( is.na(weather_2019[, names(weather_2019)[13]]), 0, 1)

In [22]:
weather_2019_arrange <- weather_2019[,c(2, 14:ncol(weather_2019))]
dim(weather_2019_arrange)
weather_2019_arrange %>% head()

[1] 365  11

,일시,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2019-01-01,1,1,1,0,0,0,1,0,0,0
2,2019-01-02,1,1,1,0,0,0,1,0,0,0
3,2019-01-03,1,1,1,0,0,0,1,0,0,0
4,2019-01-04,1,1,1,1,0,0,1,0,0,0
5,2019-01-05,1,1,1,0,0,0,1,0,0,0
6,2019-01-06,1,1,1,0,0,0,1,0,0,0


In [23]:
weather_arrange <- bind_rows(weather_2017_arrange, weather_2018_arrange, weather_2019_arrange)
dim(weather_arrange)
weather_arrange %>% head()

[1] 1095   11

,일시,평균온도,최저온도,최고온도,최대풍속,평균풍속,평균습도,평균지면온도,강수량,적설량,안개시간
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2017-01-01,0,0,0,0,1,0,0,0,0,0
2,2017-01-02,0,0,0,0,0,0,0,0,0,0
3,2017-01-03,0,0,0,0,0,0,0,0,0,0
4,2017-01-04,0,0,0,0,0,0,0,0,0,0
5,2017-01-05,0,0,0,0,0,0,0,0,0,0
6,2017-01-06,0,0,0,0,0,0,0,0,0,0


In [24]:
write.csv(weather_arrange, file='weather_arrange.csv', row.names=F)